<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/---PDF-20250403/blob/main/_note__20250412.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#【note公開】小説家になろうデータ抽出ツール（簡易通常版のみ）_20250412.ipynb

In [ ]:
import requests
import pandas as pd
from IPython.display import display, HTML
import time
import urllib.parse

def search_narou_novels(keyword='', biggenre=1, limit=20, start=1, order='new'):
    """
    なろう小説APIを使って恋愛系の小説をキーワード検索する

    Parameters:
    -----------
    keyword : str
        検索したいキーワード
    biggenre : int
        大ジャンル (1=恋愛, 2=ファンタジー, 3=文芸, 4=SF, 99=その他, 98=ノンジャンル)
    limit : int
        取得する小説の数 (最大500)
    start : int
        取得開始位置
    order : str
        並び順 (new=新着更新順, favnovelcnt=ブックマーク数順, etc.)

    Returns:
    --------
    pd.DataFrame
        検索結果をDataFrameで返す
    """
    # URLエンコード
    encoded_keyword = urllib.parse.quote(keyword, encoding='utf-8')

    # APIのベースURL
    base_url = "https://api.syosetu.com/novelapi/api/"

    # パラメータ設定
    params = {
        "out": "json",
        "word": encoded_keyword,
        "biggenre": biggenre,
        "lim": limit,
        "st": start,
        "order": order,
    }

    # URLパラメータを作成
    url_params = "&".join([f"{k}={v}" for k, v in params.items() if v])
    url = f"{base_url}?{url_params}"

    print(f"リクエストURL: {url}")

    try:
        # APIリクエスト
        response = requests.get(url)
        response.raise_for_status()

        # JSONデータを取得
        data = response.json()

        # 最初の要素は全作品数なので除外
        all_count = data[0]["allcount"]
        novels = data[1:]

        print(f"全作品数: {all_count}")

        if not novels:
            print("検索結果が見つかりませんでした。")
            return pd.DataFrame()

        # DataFrameに変換
        df = pd.DataFrame(novels)

        # 表示用に列を選択・整形
        if not df.empty:
            # 必要な列だけを選択
            columns_to_display = [
                "title", "writer", "story", "keyword",
                "genre", "ncode", "general_lastup", "length"
            ]

            # 存在する列だけを選択
            available_columns = [col for col in columns_to_display if col in df.columns]
            df_display = df[available_columns]

            # ジャンルコードを名前に変換
            genre_map = {
                101: "異世界〔恋愛〕",
                102: "現実世界〔恋愛〕"
            }

            if "genre" in df_display.columns:
                df_display["genre"] = df_display["genre"].map(lambda x: genre_map.get(x, str(x)))

            # Nコードにリンクを追加
            if "ncode" in df_display.columns:
                df_display["ncode"] = df_display["ncode"].map(
                    lambda x: f'<a href="https://ncode.syosetu.com/{x}/" target="_blank">{x}</a>'
                )

            return df_display

        return df

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return pd.DataFrame()

def display_with_clickable_links(df):
    """DataFrameをHTMLで表示し、リンクをクリック可能にする"""
    display(HTML(df.to_html(escape=False)))

def search_and_display():
    """検索して結果を表示する対話型関数"""
    # 検索条件の入力
    keyword = input("検索キーワードを入力してください: ")

    # ジャンル選択
    print("\n大ジャンルを選択してください:")
    print("1: 恋愛 (デフォルト)")
    print("2: ファンタジー")
    print("3: 文芸")
    print("4: SF")
    print("99: その他")
    print("98: ノンジャンル")

    try:
        biggenre = int(input("番号を入力してください (デフォルト:1): ") or "1")
    except ValueError:
        biggenre = 1

    # 並び順選択
    print("\n並び順を選択してください:")
    print("new: 新着更新順 (デフォルト)")
    print("favnovelcnt: ブックマーク数の多い順")
    print("reviewcnt: レビュー数の多い順")
    print("hyoka: 総合ポイントの高い順")
    print("dailypoint: 日間ポイントの高い順")
    print("weeklypoint: 週間ポイントの高い順")
    print("monthlypoint: 月間ポイントの高い順")

    order = input("並び順を入力してください (デフォルト:new): ") or "new"

    # 取得件数
    try:
        limit = int(input("\n取得する件数を入力してください (最大500, デフォルト:20): ") or "20")
        limit = min(500, max(1, limit))  # 1～500の範囲に制限
    except ValueError:
        limit = 20

    # 検索実行
    print("\n検索中...")
    df = search_narou_novels(keyword, biggenre, limit, 1, order)

    if not df.empty:
        # 結果の表示
        print("\n検索結果:")
        display_with_clickable_links(df)
    else:
        print("検索結果がありませんでした。")

# 使用例
if __name__ == "__main__":
    search_and_display()

# 簡単な使用例
# df = search_narou_novels("学園", biggenre=1, limit=10)
# display_with_clickable_links(df)

検索キーワードを入力してください: 吸血鬼　恋愛

大ジャンルを選択してください:
1: 恋愛 (デフォルト)
2: ファンタジー
3: 文芸
4: SF
99: その他
98: ノンジャンル
番号を入力してください (デフォルト:1): 1

並び順を選択してください:
new: 新着更新順 (デフォルト)
favnovelcnt: ブックマーク数の多い順
reviewcnt: レビュー数の多い順
hyoka: 総合ポイントの高い順
dailypoint: 日間ポイントの高い順
weeklypoint: 週間ポイントの高い順
monthlypoint: 月間ポイントの高い順
並び順を入力してください (デフォルト:new): new

取得する件数を入力してください (最大500, デフォルト:20): 30

検索中...
リクエストURL: https://api.syosetu.com/novelapi/api/?out=json&word=%E5%90%B8%E8%A1%80%E9%AC%BC%E3%80%80%E6%81%8B%E6%84%9B&biggenre=1&lim=30&st=1&order=new
全作品数: 307

検索結果:


<ipython-input-1-f093b595e37f>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_display["genre"] = df_display["genre"].map(lambda x: genre_map.get(x, str(x)))
<ipython-input-1-f093b595e37f>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_display["ncode"] = df_display["ncode"].map(


,title,writer,story,keyword,genre,ncode,general_lastup,length
0,生贄、のち寵愛。～魔物たちに食べられるはずがいつの間にか大切にされてます？～,杏仁堂ふーこ,魔物たちとのほのぼのスローライフ、のはずが…？！\n両親を病で亡くし、閉鎖的な田舎の村での扱いは最底辺だったルーナ。知らない間に『生贄』にされ、村を追い出されてしまった。\n自暴自棄になって訪ねた吸血鬼の屋敷。\nそこには吸血鬼だけではなく、悪魔、魔獣までいた。\n吸血鬼・レミには「帰れ」と言われてしまったが、魔獣・ルディの「泊っていきなよ」という言葉に行くアテもないルーナは屋敷に一晩泊まることになった。\n──「僕が貰っても問題ないよね？」「ジェット、はんぶんこする？」\nルーナの知らないところで交わされていた会話。レミが血を飲もうとしないので、ルディと悪魔・ジェットはルーナを食べようと企んでいた──。\n吸血鬼、悪魔、魔獣の三人と交流を重ねていくうちに、いつの間にやら……？\n※翌日カクヨムにも投稿予定。,シリアス ほのぼの 女主人公 人外 西洋 逆ハーレム 魔法 日常 ハッピーエンド 吸血鬼 悪魔 魔獣 異種恋愛 溺愛,異世界〔恋愛〕,N8990KF,2025-04-11 20:08:41,79524
1,ヴァンパイアキス,志築いろは,ねぇお姉さん、食べていい――？\n\nヴァンパイアおよびグール退治を専門とする団体『クルースニク』。出自ゆえに誰よりもヴァンパイアを憎むエルザは、ある夜、任務先で立ち入り禁止区域を徘徊していた怪しい男に遭遇する。エルザは聴取のために彼を支部まで連れ帰るものの、男の突拍子もない言動に振り回されるばかり。\nそんなある日、事件は起こる。不審な気配をたどって地下へ降りれば、仲間の一人が全身の血を吸われて死んでいた。地下にいるのは例の男ただひとり。男の正体はヴァンパイアだったのだ。だが気づいたときにはすでに遅く、エルザ一人ではヴァンパイア相手に手も足も出ない。\n死を覚悟したのもつかの間、エルザが目覚めるとそこは、なぜか男の屋敷のベッドの上だった。\nその日を境に、エルザと屋敷の住人たちとの奇妙な共同生活が始まる。\n\nヴァンパイア×恋愛ファンタジー。\n\nこの作品は、カクヨム様、エブリスタ様、アルファポリス様にも掲載しています。,シリアス ダーク 女主人公 人外 西洋 ハッピーエンド 異類婚姻譚 ヴァンパイア 吸血鬼 執着 ヤンデレ バトル 恋愛ファンタジー ネトコン13感想 R15 残酷な描写あり,異世界〔恋愛〕,N2049KH,2025-04-11 19:40:00,25413
2,吸血鬼の神に、選ばれし、フォト姫～異類と人間世界～,Aートゥモロー,かつて…、\n吸血鬼たちと、人間（アース）が、\n戦を、起こし、平和条件として…、\n人間（アース）は、\n吸血鬼たちに、、吸血鬼の神に、\n選ばれし姫、【フォト姫】を、\n差し出し、\n\nそして…、フォト姫は、\n吸血鬼たちにも、\n愛された姫で、あった＿。\n\nそして…、また、フォト姫が、\n生まれ、新たなるフォト姫の、\n歴史が、始まる＿！！\n\n異世界恋愛、吸血鬼などの、\n異類と、×人間（アース）物語。\n\n【毎週、金曜日、公開！！】\n,R15 残酷な描写あり 異類婚姻譚 身分差 女主人公 人外 中世 恋愛 ホラー 異世界,異世界〔恋愛〕,N8750HF,2025-04-11 07:00:00,181967
3,愛の瞳のリリアック,vlad,趣味がピアノで好きな作曲家はバルトーク！な日本人の女子大生チカは、旅行でパリを訪れていた際になぜか現代から19世紀のヨーロッパにタイムスリップしてしまう。そこで偶然出会った吸血鬼の青年ミルチャに「血が美味しい」と気に入られ、ルーマニアにある古城へと連れ去られるのだった。そこには人間から隠れて暮らしている吸血鬼達と悪魔がいた。どうにかして現代に戻りたいチカだが、ミルチャに「帰さない」と宣言され、強制的に古城で生活することに。\n【仮面で顔を隠した吸血鬼ミルチャと、音楽好き女子大生チカのタイムスリップ歴史恋愛ファンタジー】\n＊のろのろマイペース更新です＊\n\n注意1）流血や傷、暴力や死の表現があります。苦手な方はご注意ください。\n注意2）この物語はフィクションです。実際の出来事、実在の人物や地名、宗教とは関係ありません。\n注意3）作者はルーマニア及びその周辺諸国が好きで書いてます。特定の国や民族を貶める意図は一切ありません。\n\n,女主人公 人外 西洋 中世 近代 ハッピーエンド ヒストリカル 吸血鬼 東欧 魔術 悪魔 ヨーロッパ 一途 執着 ファンタジー R15 残酷な描写あり,現実世界〔恋愛〕,N9877KC,2025-04-08 01:47:29,63577
4,紫陽花の日次記,HILLA,あらすじ\n高校生になったばかりの主人公、｜原田紫陽花《はらだしよか》は、カフェ巡りが大好きな女の子。\n\n可愛いという理由で前の席の女の子に声をかけて友達になり、カッコいい男の子にときめいてしまう。\n\nとても平凡な女の子が織りなすファンタジーな恋愛模様を、彼女の日記を通して知っていく物語になります。\n大好きになった彼の正体は吸血鬼だった……というお話になります。\n\n\n※4コマ漫画みたいにさくっと読める読み物があってもいいかもと思い、日記調の物語を書くことにしました。(作者が知らないだけで、すでにそういうジャンルが存在するかもです)\n\n※1日のうちで印象に残った出来事を思うままに綴る日記になりますので、1週間分を日曜の22時に投稿します。\n\n※日記になりますので、1ページの長さは決まっていません。短い週もあれば長い週も出てきます。\n\n※設定は現在ですが、まごうことなきフィクションですので、「それなんだが……」が出てくるかもしれません。フィクションで日記風の小説ですので、「それなんだが……」はご了承ください。\n\n\n\n\n,ほのぼの 女主人公 人外 学園 現代 日常 ハッピーエンド 青春 スクールラブ 恋愛 吸血鬼 高校生 R15,現実世界〔恋愛〕,N0187IX,2025-04-06 22:00:00,134827
5,ヴァンパイアの歌姫,MAINA,大人気の4人組のアイドルグループ、Mudia（ミューディア）。彼女達には、誰にも言えない秘密があった────。\n\nそれは、『純血のヴァンパイア』であること。\n\nヴァンパイアにとって純血種は、王であり逆らえない存在。純血種としての役目を果たす一方で、もう一つの使命も帯びていた。\n\nその使命とは──── 世界の秩序とバランスを保ち守護する存在、『世界の巫女』。世界に選ばれし彼女達は、ステージに立ちながら《歌》で世界に蔓延る《影》を浄化していた。\n\nアイドルの顔、巫女の使命、そしてヴァンパイアの王としての役目。\n\nそんな忙しい日々を送る中。新たな力を得る為、世界の巫女である彼女達は、それぞれ『神様』とパートナーを組むことに────！？\n\nその傍らヴァンパイアだけの世界を創りあげる為、協力を仰がれたヴァンパイアの兄妹達も活躍する。\n\n\nヴァンパイア×神様のドキドキファンタジーライフ。❀·̩͙꙳。\n\n,女主人公 魔王 学園 魔法 アイリスIF6大賞 ヴァンパイア 神 異世界 現実世界 吸血鬼 悪魔 恋愛 ファンタジー 魔法使い 精霊 R15 残酷な描写あり,異世界〔恋愛〕,N8610IV,2025-04-04 23:59:34,32425
6,猫耳は今日も愛されてる,もず,猫の妖怪と人間の血が混じった真一郎は、魔獣高校に通う高校三年生。魔獣高校はその名の通り、あらゆる妖怪が人間社会に溶け込むために建てられた学校。妖怪差別がある世の中なので、特にやりたいことが見つからず学生生活を過ごしていたが、そんな真一郎の運命を変える人物がこの春この高校に入学した――――。,日常 スクールラブ ラブコメ 吸血鬼 獣人 BL 多重人格 男の娘 妖怪 NL 多重恋愛 兄弟 R15 残酷な描写あり ボーイズラブ,異世界〔恋愛〕,N4834IL,2025-03-25 16:21:37,109822
7,吸血姫の緋唇〜氷の皇子と紡ぐ異種族恋愛譚〜,猫餅,血を糧として生きる人族吸血種。彼らが暮らすブラッドナイト王国の第三王女、フェリシアは突如として常春の国と名高いネモ帝国への留学を女王たる母から命じられた。\n本来は第二王子の兄が向かうはずだったそこへ、フェリシアは拒否権なく向かうこととなったのだ。\nそんな彼女の前に現れたのは、氷の皇子と名高いネモ帝国皇太子。だが、その呼び名とは異なり初対面のフェリシアへも常春のような笑みと態度で接してくれる人物で──。\n\n周囲には冷たく、フェリシアにだけ甘い氷の皇子×ちょっと天然で呑気な吸血種のお姫さま！\n\nカクヨム様で同時投稿、アルファポリス様とソリスピア様で順次投稿しております。,ＥＳＮ大賞８ ほのぼの 女主人公 人外 西洋 学園 中世 異世界ファンタジー 吸血鬼 異種族恋愛 溺愛 魔術,異世界〔恋愛〕,N3595KC,2025-03-11 07:10:00,78750
8,幼女吸血鬼と取り戻せない程の恋をした,菜乃音,ある日の夜、白井陽（はる）は学校で美少女と名高い少女――アリア・コーラルブラッドと出会った。\n 人ならざる深紅の瞳を持ち、容姿端麗でありながら幼女体型